<a href="https://www.kaggle.com/code/mmellinger66/s3e9-concrete-strength-models?scriptVersionId=122032392" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

 <div style="background-color:rgba(177, 156, 217, 0.6);border-radius:5px;display:fill"><h1 style="text-align: center;padding: 12px 0px 12px 0px;">Playground Season 3: Episode 9 - Concrete Strength Models</h1>
</div>

## Problem Type

Regression

## Evaluation Metric

$$RMSE = \sqrt{\frac{1}{N} \sum_{i=1}^N (y_i - \hat{y_i})^2}$$

```python
score = metrics.mean_squared_error(yvalid, preds_valid, squared=False)
```

<div style="background-color:rgba(177, 156, 217, 0.6);border-radius:5px;display:fill"><h1 style="text-align: center;padding: 12px 0px 12px 0px;">Import Libraries</h1>
</div>

In [1]:
from typing import List, Set, Dict, Tuple, Optional

import os
import time
from pathlib import Path
import glob
import gc

import pandas as pd
import numpy as np

from sklearn import impute
from sklearn import metrics
from sklearn import preprocessing
from sklearn import linear_model
from sklearn import svm
from sklearn import cluster
from sklearn import model_selection
from sklearn import ensemble
from sklearn import datasets

import xgboost as xgb
import catboost as cb
import lightgbm as lgb

import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances

from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax

# Visualization Libraries
import matplotlib as mpl
import matplotlib.pylab as plt
import seaborn as sns
import missingno as msno
from folium import Map
from folium.plugins import HeatMap
from IPython.display import display_html, display_markdown, display_latex
from colorama import Fore, Style

import warnings
warnings.filterwarnings('ignore')

pd.set_option("display.max_rows", 999)
pd.set_option("display.precision", 5)

<div style="background-color:rgba(177, 156, 217, 0.6);border-radius:5px;display:fill"><h1 style="text-align: center;padding: 12px 0px 12px 0px;">Configuration</h1>
</div>

In [2]:
TARGET="Strength"
ID="id"

# Optuna
objective_direction = "minimize"  # minimize, maximize

In [3]:
class Config:
    path:str = "../input/playground-series-s3e9/"
    load_original_data:bool = True # Some Competitions use synthetic data, based on real data
    original_data_path:str = "../input/predict-concrete-strength/ConcreteStrengthData.csv"
    gpu:bool = False
    optimize:bool = True
    n_optuna_trials:int = 100 # 5, 10, 30, 100
    fast_render:bool = False
    calc_probability:bool = False
    debug:bool = False
    seed:int = 42
    N_ESTIMATORS:int = 500  # 100, 300, 1000, 2000, 5000, 15_000, 20_000 GBDT
    GPU_N_ESTIMATORS:int = 2000 # Want models to run fast during dev
    N_FOLDS:int = 10
        

In [4]:
class clr:
    S = Style.BRIGHT + Fore.LIGHTRED_EX
    E = Style.RESET_ALL

<div style="background-color:rgba(177, 156, 217, 0.6);border-radius:5px;display:fill"><h1 style="text-align: center;padding: 12px 0px 12px 0px;">Library</h1>
</div>

In [5]:
def read_data(path: str, analyze:bool=True) -> (pd.DataFrame, pd.DataFrame, pd.DataFrame):
    data_dir = Path(path)

    train = pd.read_csv(data_dir / "train.csv")
    test = pd.read_csv(data_dir / "test.csv")
    submission_df = pd.read_csv(data_dir / "sample_submission.csv")

    if analyze:
        print(clr.S + "=== Shape of Data ==="+clr.E)
        print(f" train data: Rows={train.shape[0]}, Columns={train.shape[1]}")
        print(f" test data : Rows={test.shape[0]}, Columns={test.shape[1]}")

        print(clr.S + "\n=== Train Data: First 5 Rows ===\n"+clr.E)
        display(train.head())
        print(f"\n{clr.S}=== Train Column Names ==={clr.E}\n")
        display(train.columns)
        print(f"\n{clr.S}=== Features/Explanatory Variables ==={clr.E}\n")
        eval_features(train)
        print(f"\n{clr.S}=== Skewness ==={clr.E}\n")
        check_skew(train)
    return train, test, submission_df

def create_submission(model_name: str, target, preds, seed:int=42, nfolds:int=5) -> pd.DataFrame:
    sample_submission[target] = preds #.astype(int)

    if len(model_name) > 0:
        fname = f"submission_{model_name}_k{nfolds}_s{seed}.csv"
    else:
        fname = "submission.csv"

    sample_submission.to_csv(fname, index=False)

    return sample_submission

def show_classification_scores(ground_truth:List[int], yhat:List[int]) -> None:
    accuracy = metrics.accuracy_score(ground_truth, yhat)
    precision = metrics.precision_score(ground_truth, yhat)
    recall = metrics.recall_score(ground_truth, yhat)
    roc = metrics.roc_auc_score(ground_truth, yhat)
    f1 = metrics.f1_score(ground_truth, yhat)

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"ROC: {roc:.4f}")
    print(f"f1: {f1:.4f}")
    

def label_encoder(train:pd.DataFrame, test:pd.DataFrame, columns:List[str]) -> (pd.DataFrame, pd.DataFrame) :
    for col in columns:
        train[col] = train[col].astype(str)
        test[col] = test[col].astype(str)
        train[col] = preprocessing.LabelEncoder().fit_transform(train[col])
        test[col] = preprocessing.LabelEncoder().fit_transform(test[col])
    return train, test   

def create_strat_folds(df:pd.DataFrame, TARGET, n_folds:int=5, seed:int=42) -> pd.DataFrame:
    print(f"TARGET={TARGET}, n_folds={n_folds}, seed={seed}")
    df["fold"] = -1

    kf = model_selection.StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=seed)
    # kf = GroupKFold(n_splits=Config.N_FOLDS)
    for fold, (train_idx, valid_idx) in enumerate(kf.split(df, df[TARGET])):
        df.loc[valid_idx, "fold"] = fold

    # df.to_csv(f"train_fold{num_folds}.csv", index=False)
    return df


def create_folds(df:pd.DataFrame, n_folds:int=5, seed:int=42) -> pd.DataFrame:
    print(f"n_folds={n_folds}, seed={seed}")
    df["fold"] = -1

    kf = model_selection.KFold(n_splits=n_folds, shuffle=True, random_state=seed)

    for fold, (train_idx, valid_idx) in enumerate(kf.split(df)):
        df.loc[valid_idx, "fold"] = fold

    # df.to_csv(f"train_fold{num_folds}.csv", index=False)
    return df

def show_fold_scores(scores: List[float]) -> (float, float):
    cv_score = np.mean(scores)  # Used in filename
    std_dev = np.std(scores)
    print(
        f"Scores -> Adjusted: {np.mean(scores) - np.std(scores):.8f} , mean: {np.mean(scores):.8f}, std: {np.std(scores):.8f}"
    )
    return cv_score, std_dev


def feature_distribution_types(df:pd.DataFrame, display:bool=True) -> (List[str], List[str]):
    continuous_features = list(df.select_dtypes(include=['int64', 'float64', 'uint8']).columns)
    categorical_features = list(df.select_dtypes(include=['object', 'bool']).columns)
    if display:
        print(f"{clr.S}Continuous Features={continuous_features}{clr.E}\n")
        print(f"{clr.S}Categorical Features={categorical_features}{clr.E}")
    return continuous_features, categorical_features   

def show_cardinality(df:pd.DataFrame, features:List[str]) -> None:
    print("=== Cardinality ===")
    print(df[features].nunique())

## === Model Support ===    

from scipy.stats import mode


def merge_test_predictions(final_test_predictions:List[float], calc_probability:bool=True) -> List[float]:

    if calc_probability:
        print("Mean")
        result = np.mean(np.column_stack(final_test_predictions), axis=1)
    else:
        print("Mode")
        mode_result = mode(np.column_stack(final_test_predictions), axis=1)
        result = mode_result[0].ravel()

    return result

def summary_statistics(X:pd.DataFrame, enhanced=True) -> None:
    desc = X.describe()
    if enhanced:
        desc.loc["var"] = X.var(numeric_only=True).tolist()
        desc.loc["skew"] = X.skew(numeric_only=True).tolist()
        desc.loc["kurt"] = X.kurtosis(numeric_only=True).tolist()

    with pd.option_context("display.precision", 2):
        style = desc.transpose().style.background_gradient(
            cmap="coolwarm"
        )  # .set_precision(4)
    display(style)
    
def show_missing_features(df:pd.DataFrame) -> None:
    missing_vals = df.isna().sum().sort_values(ascending=False)
    print(missing_vals[missing_vals > 0])


def show_duplicate_records(df:pd.DataFrame) -> None:
    dups = df.duplicated()
    print(dups.sum())


def eval_features(df:pd.DataFrame) -> (List[str], List[str], List[str]):
    ## Separate Categorical and Numerical Features
    categorical_features = list(
        df.select_dtypes(include=["category", "object"]).columns
    )
    continuous_features = list(df.select_dtypes(include=["number"]).columns)

    print(f"{clr.S}Continuous features:{clr.E} {continuous_features}")
    print(f"{clr.S}Categorical features:{clr.E} {categorical_features}")
    print("\n --- Cardinality of Categorical Features ---\n")

    for feature in categorical_features:
        cardinality = df[feature].nunique()
        if cardinality < 10:
            print(f"{clr.S}{feature}{clr.E}: cardinality={cardinality}, {df[feature].unique()}")
        else:
            print(f"{clr.S}{feature}{clr.E}: cardinality={cardinality}")
    all_features = categorical_features + continuous_features
    return all_features, categorical_features, continuous_features


def show_feature_importance(feature_importance_lst:List[str]) -> None:
    fis_df = pd.concat(feature_importance_lst, axis=1)

    fis_df.sort_values("0_importance", ascending=True).head(40).plot(
        kind="barh", figsize=(12, 12), title="Feature Importance Across Folds"
    )
    plt.show()


def show_feature_target_crosstab(df:pd.DataFrame, feature_lst:List[str], target:str) -> None:
    for feature in feature_lst:
        print(f"\n=== {feature} vs {target} ===\n")
        display(
            pd.crosstab(df[feature], df[target], margins=True)
        )  # display keeps bold formatting


def show_cardinality(df:pd.DataFrame, features:List[str]) -> None:
    print(f"{clr.S}=== Cardinality ==={clr.E}")
    print(df[features].nunique())


def show_unique_features(df:pd.DataFrame, features:List[str]) -> None:
    for col in features:
        print(col, sorted(df[col].dropna().unique()))


def feature_distribution_types(df:pd.DataFrame, display:bool=True) -> (List[str], List[str]):
    continuous_features = list(
        df.select_dtypes(include=["int64", "float64", "uint8"]).columns
    )
    categorical_features = list(df.select_dtypes(include=["object", "bool"]).columns)
    if display:
        print(f"{clr.S}Continuous Features={clr.E}{continuous_features}\n")
        print(f"{clr.S}Categorical Features={clr.E}{categorical_features}")
    return continuous_features, categorical_features


def describe(X:pd.DataFrame) -> None:
    """Deprecated: Use summary_statistics()"""
    desc = X.describe()
    desc.loc['var'] = X.var(numeric_only=True).tolist()
    desc.loc['skew'] = X.skew(numeric_only=True).tolist()
    desc.loc['kurt'] = X.kurtosis(numeric_only=True).tolist()

    with pd.option_context('display.precision', 2):
        style = desc.transpose().style.background_gradient(cmap='coolwarm') #.set_precision(4)
    display(style)
  

def check_skew(df:pd.DataFrame) -> None:
    skew = df.skew(skipna=True,numeric_only=True).sort_values(ascending=False)
    print(skew)
    
def gpu_ify_lgbm(lgbm_dict):
    if Config.gpu:
        lgbm_dict["device"] = "gpu"
        lgbm_dict["boosting_type"] = "gbdt"
        lgbm_dict["gpu_platform_id"] = 0
        lgbm_dict["gpu_device_id"] = 0
    return lgbm_dict

def gpu_ify_cb(params):
    if Config.gpu:
        params["task_type"] = "GPU"
    return params    


<div style="background-color:rgba(177, 156, 217, 0.6);border-radius:5px;display:fill"><h1 style="text-align: center;padding: 12px 0px 12px 0px;">Optuna Hyperparameter Optimization Library</h1>
</div>

In [6]:
def objective_xgb(trial, X_train, X_valid, y_train, y_valid):

    xgb_params = {
        #         "objective": trial.suggest_categorical("objective", ["multi:softmax"]),
        #         "eval_metric": "mlogloss",
        #         "objective": "multi:softmax",
#         "objective": trial.suggest_categorical("objective", ["mae", "rmse"]),

        "eval_metric": "rmse",  # auc, rmse, mae
        "objective": "reg:squarederror", # Normal Distribution
#         "objective": "reg:gamma", # Gamma Distribution

        #         "enable_categorical": trial.suggest_categorical("use_label_encoder", [True]),
        "use_label_encoder": trial.suggest_categorical("use_label_encoder", [False]),
        "n_estimators": trial.suggest_int("n_estimators", 100, 5000, 100),
        "learning_rate": trial.suggest_loguniform("learning_rate", 1e-2, 0.25),
        "subsample": trial.suggest_float("subsample", 0.1, 1, step=0.01),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.05, 1, step=0.01),
        "max_depth": trial.suggest_int("max_depth", 1, 20),  # 10
        "gamma": trial.suggest_float("gamma", 0, 100, step=0.1),
        "booster": trial.suggest_categorical("booster", ["gbtree"]),
        "tree_method": trial.suggest_categorical(
            "tree_method", ["hist"]
        ),  # hist, gpu_hist
#         "predictor": "gpu_predictor",
        "reg_lambda": trial.suggest_loguniform("reg_lambda", 1e-8, 100),
        "reg_alpha": trial.suggest_loguniform("reg_alpha", 1e-8, 100),
        "random_state": trial.suggest_categorical("random_state", [42]),
        "n_jobs": trial.suggest_categorical("n_jobs", [4]),
        "min_child_weight": trial.suggest_loguniform("min_child_weight", 1e-1, 1e3),
        # "min_child_weight": trial.suggest_categorical("min_child_weight", [256]),
        
    }

    # Model loading and training
    model = xgb.XGBRegressor(**xgb_params)
    model.fit(
        X_train,
        y_train,
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
        early_stopping_rounds=500,
        verbose=0,
    )

    print(f"Number of boosting rounds: {model.best_iteration}")
    #     oof = model.predict_proba(X_valid)[:, 1] # Probability
    oof = model.predict(X_valid)  # Classification: 0,1

    return metrics.mean_squared_error(y_valid, oof, squared=False)


def objective_lgbm(trial, X_train, X_valid, y_train, y_valid):

    lgbm_params = {
        "objective": trial.suggest_categorical("objective", ["mae", "rmse"]),
        #         "n_estimators": trial.suggest_categorical("n_estimators", [1_000]),
        #         "n_estimators": trial.suggest_categorical("n_estimators", [5000]),
        "n_estimators": trial.suggest_int("n_estimators", 700, 5000),
        "importance_type": "gain",
        "reg_alpha": trial.suggest_loguniform("reg_alpha", 1e-8, 10.0),
        "reg_lambda": trial.suggest_loguniform("reg_lambda", 1e-8, 10.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.05, 1, step=0.01),
        "num_leaves": trial.suggest_int("num_leaves", 2, 1000),
        "feature_fraction": trial.suggest_uniform("feature_fraction", 0.1, 1.0),
        "bagging_fraction": trial.suggest_uniform("bagging_fraction", 0.1, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 0, 15),
        "min_child_samples": trial.suggest_int("min_child_samples", 1, 300),
        "subsample": trial.suggest_float("subsample", 0.1, 1, step=0.01),
        "learning_rate": trial.suggest_loguniform("learning_rate", 1e-2, 0.25),
        "max_depth": trial.suggest_int("max_depth", 1, 100),
        "random_state": trial.suggest_categorical("random_state", [42]),
        "n_jobs": trial.suggest_categorical("n_jobs", [4]),
        #         'min_child_weight': trial.suggest_loguniform('min_child_weight', 1e-1, 1e3),
        # "min_child_weight": trial.suggest_categorical("min_child_weight", [256]),
    }

    # Model loading and training
    model = lgb.LGBMRegressor(**lgbm_params)
    model.fit(
        X_train,
        y_train,
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
        eval_metric="mae",
        callbacks=[
            lgb.log_evaluation(500),
            lgb.early_stopping(500, False, True),
        ],
    )

    #     print(f"Number of boosting rounds: {model.best_iteration}")
    oof = model.predict(X_valid)

    return metrics.mean_squared_error(y_valid, oof, squared=False)
#     return metrics.mean_absolute_error(y_valid, oof)


def objective_clf_lgbm(trial, X_train, X_valid, y_train, y_valid):

    params = {
        "boosting_type": "gbdt",
        # "objective": trial.suggest_categorical("objective", ["mae", "rmse"]),
        #         "objective": trial.suggest_categorical("objective", ["multi:softprob"]),
        #         "n_estimators": trial.suggest_categorical("n_estimators", [1_000]),
        #         "n_estimators": trial.suggest_categorical("n_estimators", [5000]),
        "n_estimators": trial.suggest_int("n_estimators", 700, 1000),
        "importance_type": "gain",
        "reg_alpha": trial.suggest_loguniform("reg_alpha", 1e-8, 10.0),
        "reg_lambda": trial.suggest_loguniform("reg_lambda", 1e-8, 10.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.05, 1, step=0.01),
        "num_leaves": trial.suggest_int("num_leaves", 2, 1000),
        "feature_fraction": trial.suggest_uniform("feature_fraction", 0.1, 1.0),
        "bagging_fraction": trial.suggest_uniform("bagging_fraction", 0.1, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 0, 15),
        "min_child_samples": trial.suggest_int("min_child_samples", 1, 300),
        "subsample": trial.suggest_float("subsample", 0.1, 1, step=0.01),
        "learning_rate": trial.suggest_loguniform("learning_rate", 1e-2, 0.25),
        "max_depth": trial.suggest_int("max_depth", 1, 100),
        "random_state": trial.suggest_categorical("random_state", [42]),
        "n_jobs": trial.suggest_categorical("n_jobs", [4]),
        #         'min_child_weight': trial.suggest_loguniform('min_child_weight', 1e-1, 1e3),
        # "min_child_weight": trial.suggest_categorical("min_child_weight", [256]),
    }
    if Config.gpu:
        params["device_type"] = "gpu"

    # Model loading and training
    model = lgb.LGBMClassifier(**params)
    model.fit(
        X_train,
        y_train,
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
        # eval_metric="mae",
        callbacks=[
            lgb.log_evaluation(500),
            lgb.early_stopping(500, False, True),
        ],
    )

    #     print(f"Number of boosting rounds: {model.best_iteration}")
    oof = model.predict(X_valid)

    #     return accuracy_score(y_valid, oof)
    return metrics.roc_auc_score(y_valid, oof)


def objective_cb(trial, X_train, X_valid, y_train, y_valid):

    cb_params = {
        "iterations": 100,
        "learning_rate": trial.suggest_loguniform("learning_rate", 0.1, 1.0),
        "l2_leaf_reg": trial.suggest_loguniform("l2_leaf_reg", 1, 100),
        "bagging_temperature": trial.suggest_loguniform(
            "bagging_temperature", 0.1, 20.0
        ),
        "random_strength": trial.suggest_float("random_strength", 1.0, 2.0),
        "depth": trial.suggest_int("depth", 1, 10),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 300),
          "use_best_model": True,
#         "task_type": "GPU",
        "random_seed": 42,
    }

    # Model loading and training
    #  model = CatBoostClassifier(**cb_params)
    model = cb.CatBoostRegressor(**cb_params)

    model.fit(
        X_train,
        y_train,
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
        # eval_metric="accuracy",
        early_stopping_rounds=500,
        verbose=False,
    )

#     print(f"Number of boosting rounds: {model.best_iteration}")
    # oof = model.predict_proba(X_valid)[:, 1]
    oof = model.predict(X_valid)  # Classification
    return metrics.mean_squared_error(y_valid, oof, squared=False)
#     return metrics.mean_absolute_error(y_valid, oof)
# 
#     return accuracy_score(y_valid, oof)

def objective_clf_cb(trial, X_train, X_valid, y_train, y_valid):

    cb_params = {
        "iterations": 10,  # 1000
        "learning_rate": trial.suggest_loguniform("learning_rate", 0.1, 1.0),
        "l2_leaf_reg": trial.suggest_loguniform("l2_leaf_reg", 1, 100),
        "bagging_temperature": trial.suggest_loguniform(
            "bagging_temperature", 0.1, 20.0
        ),
        "random_strength": trial.suggest_float("random_strength", 1.0, 2.0),
        "depth": trial.suggest_int("depth", 1, 10),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 300),
        "use_best_model": True,
#             "task_type": "GPU",
        "random_seed": 42,
    }

    # Model loading and training
    model = cb.CatBoostClassifier(**cb_params)
    model.fit(
        X_train,
        y_train,
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
        # eval_metric="accuracy",
        early_stopping_rounds=500,
        verbose=False,
    )

    # print(f"Number of boosting rounds: {model.best_iteration}")
    # oof = model.predict_proba(X_valid)[:, 1]
    oof = model.predict(X_valid)  # Classification

    return metrics.accuracy_score(y_valid, oof)

<div style="background-color:rgba(177, 156, 217, 0.6);border-radius:5px;display:fill"><h1 style="text-align: center;padding: 12px 0px 12px 0px;">Load Train/Test Data and Analyze</h1>
</div>

## Load the following files

 - train.csv - Data used to build our machine learning model
 - test.csv - Data used to build our machine learning model. Does not contain the target variable
 - sample_submission.csv - A file in the proper format to submit test predictions

In [7]:
%%time
train, test, sample_submission = read_data(Config.path, analyze=True)                                

=== Shape of Data ===
 train data: Rows=5407, Columns=10
 test data : Rows=3605, Columns=9

=== Train Data: First 5 Rows ===



,id,CementComponent,BlastFurnaceSlag,FlyAshComponent,WaterComponent,SuperplasticizerComponent,CoarseAggregateComponent,FineAggregateComponent,AgeInDays,Strength
0,0,525.0,0.0,0.0,186.0,0.0,1125.0,613.0,3,10.38
1,1,143.0,169.0,143.0,191.0,8.0,967.0,643.0,28,23.52
2,2,289.0,134.7,0.0,185.7,0.0,1075.0,795.3,28,36.96
3,3,304.0,76.0,0.0,228.0,0.0,932.0,670.0,365,39.05
4,4,157.0,236.0,0.0,192.0,0.0,935.4,781.2,90,74.19



=== Train Column Names ===



Index(['id', 'CementComponent', 'BlastFurnaceSlag', 'FlyAshComponent',
       'WaterComponent', 'SuperplasticizerComponent',
       'CoarseAggregateComponent', 'FineAggregateComponent', 'AgeInDays',
       'Strength'],
      dtype='object')


=== Features/Explanatory Variables ===

Continuous features: ['id', 'CementComponent', 'BlastFurnaceSlag', 'FlyAshComponent', 'WaterComponent', 'SuperplasticizerComponent', 'CoarseAggregateComponent', 'FineAggregateComponent', 'AgeInDays', 'Strength']
Categorical features: []

 --- Cardinality of Categorical Features ---


=== Skewness ===

AgeInDays                    2.74687
SuperplasticizerComponent    1.41169
FlyAshComponent              1.30469
BlastFurnaceSlag             1.12120
Strength                     0.38073
CementComponent              0.34128
id                           0.00000
CoarseAggregateComponent    -0.08145
WaterComponent              -0.21528
FineAggregateComponent      -0.44738
dtype: float64
CPU times: user 51.5 ms, sys: 6.93 ms, total: 58.4 ms
Wall time: 94.7 ms


In [8]:
train.head()

,id,CementComponent,BlastFurnaceSlag,FlyAshComponent,WaterComponent,SuperplasticizerComponent,CoarseAggregateComponent,FineAggregateComponent,AgeInDays,Strength
0,0,525.0,0.0,0.0,186.0,0.0,1125.0,613.0,3,10.38
1,1,143.0,169.0,143.0,191.0,8.0,967.0,643.0,28,23.52
2,2,289.0,134.7,0.0,185.7,0.0,1075.0,795.3,28,36.96
3,3,304.0,76.0,0.0,228.0,0.0,932.0,670.0,365,39.05
4,4,157.0,236.0,0.0,192.0,0.0,935.4,781.2,90,74.19


In [9]:
def load_original_data(path:str) -> pd.DataFrame:
#     original = pd.read_csv(path, index_col=[0])
    original = pd.read_csv(path)

    original = original.reset_index()
    original['id'] = original['index'] + 100000
    original = original.drop(columns = ['index'])
    original = original.rename(columns = {'CementComponent ':'CementComponent'})
    original.set_index('id', inplace=True)
#     original = original[-original.depth.isna()]
    print(f"Shape={original.shape}")
    return original
#     original.head()

if Config.load_original_data:    
    original = load_original_data(Config.original_data_path)
    display(original.head())

Shape=(1030, 9)


,CementComponent,BlastFurnaceSlag,FlyAshComponent,WaterComponent,SuperplasticizerComponent,CoarseAggregateComponent,FineAggregateComponent,AgeInDays,Strength
id,,,,,,,,,
100000,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
100001,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
100002,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
100003,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
100004,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [10]:
if Config.load_original_data:
    train['is_original']    = 0
    test['is_original']     = 0
    original['is_original'] = 1
#     combined = pd.concat([train, original], ignore_index=True) #.drop_duplicates()
    combined = pd.concat([train, original])

    train = combined
#     combined.head()
    print(f"Shape={combined.shape}")


Shape=(6437, 11)


In [11]:
summary_statistics(train.drop(columns=[ID], axis=1), enhanced=True)

,count,mean,std,min,25%,50%,75%,max,var,skew,kurt
CementComponent,6437.00,296.29,105.57,102.00,212.50,295.80,374.00,540.00,11145.39,0.37,-0.55
BlastFurnaceSlag,6437.00,61.06,84.06,0.00,0.00,0.00,129.90,359.40,7066.49,1.06,-0.11
FlyAshComponent,6437.00,35.44,56.80,0.00,0.00,0.00,94.00,200.10,3226.28,1.16,-0.28
WaterComponent,6437.00,184.51,19.04,121.80,173.00,186.00,192.00,247.00,362.59,-0.17,0.67
SuperplasticizerComponent,6437.00,4.44,5.79,0.00,0.00,0.00,8.70,32.20,33.51,1.30,1.92
CoarseAggregateComponent,6437.00,988.95,77.56,801.00,938.00,975.60,1047.00,1145.00,6014.85,-0.08,-0.56
FineAggregateComponent,6437.00,771.60,78.96,594.00,734.30,781.00,821.00,992.60,6234.31,-0.41,-0.02
AgeInDays,6437.00,50.78,68.99,1.00,7.00,28.00,56.00,365.00,4759.62,2.82,8.76
Strength,6437.00,35.51,16.45,2.33,23.69,33.96,45.85,82.60,270.61,0.39,-0.35
is_original,6437.00,0.16,0.37,0.00,0.00,0.00,0.00,1.00,0.13,1.86,1.44


## Outlier Detection

In [12]:
# https://www.kaggle.com/code/lyasdemir/best-algorithm-for-prediction-xgboost
    
def iqr(data:pd.DataFrame, var:str):# outliers detecion .
    q1 = np.quantile(data[var], 0.25)
    q3 = np.quantile(data[var], 0.75)
    diff = q3 - q1
    lower_t = q1 - (1.5 * diff)
    upper_t = q3 + (1.5 * diff)
    return data[(data[var] < lower_t) | (data[var] > upper_t)]

# iqr(train, "squareMeters")

In [13]:
# https://www.kaggle.com/code/sujithmandala/playground-s3-e8-ensemble-model-98-accuracy
    
def detect_outliers(data:pd.DataFrame) -> pd.DataFrame:
    outlier_percents = {}
    for column in data.columns:
        if data[column].dtype != object:
            q1 = np.quantile(data[column], 0.25)
            q3 = np.quantile(data[column], 0.75)
            iqr = q3 - q1
            upper_bound = q3 + (1.5 * iqr)
            lower_bound = q1 - (1.5 * iqr)
            outliers = data[(data[column] > upper_bound) | (data[column] < lower_bound)][column]
            outlier_percentage = len(outliers) / len(data[column]) * 100
            outlier_percents[column] = outlier_percentage
            outlier_dataframe = pd.DataFrame(data = outlier_percents.values() ,index=outlier_percents.keys() ,columns=['Outlier_percentage'])
    
    return outlier_dataframe.sort_values(by = 'Outlier_percentage', ascending = False)

detect_outliers(test)

,Outlier_percentage
FineAggregateComponent,8.54369
WaterComponent,8.26630
AgeInDays,7.93343
SuperplasticizerComponent,1.47018
BlastFurnaceSlag,0.41609
id,0.00000
CementComponent,0.00000
FlyAshComponent,0.00000
CoarseAggregateComponent,0.00000
is_original,0.00000


<div style="background-color:rgba(177, 156, 217, 0.6);border-radius:5px;display:fill"><h1 style="text-align: center;padding: 12px 0px 12px 0px;">Feature Engineering</h1>
</div>

## Categorical/Numerical Variables

## Handle Outliers
- https://www.kaggle.com/code/lyasdemir/best-algorithm-for-prediction-xgboost
- https://www.kaggle.com/code/mnokno/paris-housing-price-prediction-using-xgboost

In [14]:
#https://www.kaggle.com/code/mmellinger66/s3e9-concrete-strength-models

def add_features(df):
#     df['Water_Cement'] = df['WaterComponent'] / df['CementComponent']
    df['Coarse_Fine'] = df['CoarseAggregateComponent'] / df['FineAggregateComponent']
    df['Aggregate'] = df['CoarseAggregateComponent'] + df['FineAggregateComponent']
#     df['Aggregate_Cement'] = df['Aggregate'] / df['CementComponent']
    df['Slag_Cement'] = df['BlastFurnaceSlag'] / df['CementComponent']
    df['Ash_Cement'] = df['FlyAshComponent'] / df['CementComponent']
    df['Plastic_Cement'] = df['SuperplasticizerComponent'] / df['CementComponent']
    df['Age_Water'] = df['AgeInDays'] / df['WaterComponent']
    return df

train = add_features(train)
test = add_features(test)

In [15]:
# features_with_outliers = []

In [16]:
# https://www.kaggle.com/code/mnokno/paris-housing-price-prediction-using-xgboost

def remove_outliers(df:pd.DataFrame) -> pd.DataFrame:
    for c in features_with_outliers:
        if c == 'garage':
            first_percentile = df[c].quantile(0.001)
            df = df[df[c] > first_percentile]

        ninety_ninth_percentile = df[c].quantile(0.999)
        df = df[df[c] < ninety_ninth_percentile]
        #df_t = df_t[(df_t[c] > first_percentile) & (df_t[c] < ninety_ninth_percentile)]
    return df


In [17]:
# print(f'Before: {len(train)}')
# train = remove_outliers(train)
# print(f'After: {len(train)}')

In [18]:
train.head(10)

,id,CementComponent,BlastFurnaceSlag,FlyAshComponent,WaterComponent,SuperplasticizerComponent,CoarseAggregateComponent,FineAggregateComponent,AgeInDays,Strength,is_original,Coarse_Fine,Aggregate,Slag_Cement,Ash_Cement,Plastic_Cement,Age_Water
0,0.0,525.0,0.0,0.0,186.0,0.0,1125.0,613.0,3,10.38,0,1.83524,1738.0,0.00000,0.0,0.00000,0.01613
1,1.0,143.0,169.0,143.0,191.0,8.0,967.0,643.0,28,23.52,0,1.50389,1610.0,1.18182,1.0,0.05594,0.14660
2,2.0,289.0,134.7,0.0,185.7,0.0,1075.0,795.3,28,36.96,0,1.35169,1870.3,0.46609,0.0,0.00000,0.15078
3,3.0,304.0,76.0,0.0,228.0,0.0,932.0,670.0,365,39.05,0,1.39104,1602.0,0.25000,0.0,0.00000,1.60088
4,4.0,157.0,236.0,0.0,192.0,0.0,935.4,781.2,90,74.19,0,1.19739,1716.6,1.50318,0.0,0.00000,0.46875
5,5.0,350.0,0.0,0.0,203.0,0.0,1055.0,775.0,7,37.43,0,1.36129,1830.0,0.00000,0.0,0.00000,0.03448
6,6.0,135.7,203.5,0.0,185.7,0.0,1076.2,759.3,28,35.10,0,1.41736,1835.5,1.49963,0.0,0.00000,0.15078
7,7.0,332.5,142.5,0.0,228.0,0.0,932.0,594.0,28,45.94,0,1.56902,1526.0,0.42857,0.0,0.00000,0.12281
8,8.0,322.0,0.0,0.0,203.0,0.0,974.0,800.0,180,42.14,0,1.21750,1774.0,0.00000,0.0,0.00000,0.88670
9,9.0,133.0,200.0,0.0,192.0,0.0,927.4,839.2,3,6.94,0,1.10510,1766.6,1.50376,0.0,0.00000,0.01562


In [19]:
train = train.reset_index(drop=True).copy()
train.head(10)

,id,CementComponent,BlastFurnaceSlag,FlyAshComponent,WaterComponent,SuperplasticizerComponent,CoarseAggregateComponent,FineAggregateComponent,AgeInDays,Strength,is_original,Coarse_Fine,Aggregate,Slag_Cement,Ash_Cement,Plastic_Cement,Age_Water
0,0.0,525.0,0.0,0.0,186.0,0.0,1125.0,613.0,3,10.38,0,1.83524,1738.0,0.00000,0.0,0.00000,0.01613
1,1.0,143.0,169.0,143.0,191.0,8.0,967.0,643.0,28,23.52,0,1.50389,1610.0,1.18182,1.0,0.05594,0.14660
2,2.0,289.0,134.7,0.0,185.7,0.0,1075.0,795.3,28,36.96,0,1.35169,1870.3,0.46609,0.0,0.00000,0.15078
3,3.0,304.0,76.0,0.0,228.0,0.0,932.0,670.0,365,39.05,0,1.39104,1602.0,0.25000,0.0,0.00000,1.60088
4,4.0,157.0,236.0,0.0,192.0,0.0,935.4,781.2,90,74.19,0,1.19739,1716.6,1.50318,0.0,0.00000,0.46875
5,5.0,350.0,0.0,0.0,203.0,0.0,1055.0,775.0,7,37.43,0,1.36129,1830.0,0.00000,0.0,0.00000,0.03448
6,6.0,135.7,203.5,0.0,185.7,0.0,1076.2,759.3,28,35.10,0,1.41736,1835.5,1.49963,0.0,0.00000,0.15078
7,7.0,332.5,142.5,0.0,228.0,0.0,932.0,594.0,28,45.94,0,1.56902,1526.0,0.42857,0.0,0.00000,0.12281
8,8.0,322.0,0.0,0.0,203.0,0.0,974.0,800.0,180,42.14,0,1.21750,1774.0,0.00000,0.0,0.00000,0.88670
9,9.0,133.0,200.0,0.0,192.0,0.0,927.4,839.2,3,6.94,0,1.10510,1766.6,1.50376,0.0,0.00000,0.01562


In [20]:
excluded_features = [TARGET, ID, "fold", "is_original"]

In [21]:
cont_features, cat_features = feature_distribution_types(train, display=True)
show_cardinality(train, cat_features)

cont_features = [feature for feature in cont_features if feature not in excluded_features]
cat_features = [feature for feature in cat_features if feature not in excluded_features]

FEATURES = cont_features + cat_features
FEATURES

Continuous Features=['id', 'CementComponent', 'BlastFurnaceSlag', 'FlyAshComponent', 'WaterComponent', 'SuperplasticizerComponent', 'CoarseAggregateComponent', 'FineAggregateComponent', 'AgeInDays', 'Strength', 'is_original', 'Coarse_Fine', 'Aggregate', 'Slag_Cement', 'Ash_Cement', 'Plastic_Cement', 'Age_Water']

Categorical Features=[]
=== Cardinality ===
Series([], dtype: float64)


['CementComponent',
 'BlastFurnaceSlag',
 'FlyAshComponent',
 'WaterComponent',
 'SuperplasticizerComponent',
 'CoarseAggregateComponent',
 'FineAggregateComponent',
 'AgeInDays',
 'Coarse_Fine',
 'Aggregate',
 'Slag_Cement',
 'Ash_Cement',
 'Plastic_Cement',
 'Age_Water']

In [22]:
train, test = label_encoder(train, test, cat_features)
# train = pd.get_dummies(train,columns=[]) # Will remove original feature names
# test = pd.get_dummies(test,columns=[])

In [23]:
train.head()

,id,CementComponent,BlastFurnaceSlag,FlyAshComponent,WaterComponent,SuperplasticizerComponent,CoarseAggregateComponent,FineAggregateComponent,AgeInDays,Strength,is_original,Coarse_Fine,Aggregate,Slag_Cement,Ash_Cement,Plastic_Cement,Age_Water
0,0.0,525.0,0.0,0.0,186.0,0.0,1125.0,613.0,3,10.38,0,1.83524,1738.0,0.00000,0.0,0.00000,0.01613
1,1.0,143.0,169.0,143.0,191.0,8.0,967.0,643.0,28,23.52,0,1.50389,1610.0,1.18182,1.0,0.05594,0.14660
2,2.0,289.0,134.7,0.0,185.7,0.0,1075.0,795.3,28,36.96,0,1.35169,1870.3,0.46609,0.0,0.00000,0.15078
3,3.0,304.0,76.0,0.0,228.0,0.0,932.0,670.0,365,39.05,0,1.39104,1602.0,0.25000,0.0,0.00000,1.60088
4,4.0,157.0,236.0,0.0,192.0,0.0,935.4,781.2,90,74.19,0,1.19739,1716.6,1.50318,0.0,0.00000,0.46875


In [24]:
# if Config.debug:
#     train = train.sample(frac=0.4, random_state=Config.seed)
#     train.reset_index()
#     print(f"Debug Shape: {train.shape}")

<div style="background-color:rgba(177, 156, 217, 0.6);border-radius:5px;display:fill"><h1 style="text-align: center;padding: 12px 0px 12px 0px;">Optuna Hyperparameter Optimization</h1>
</div>

In [25]:
%%time

if Config.optimize:
    y = train[TARGET]
    X = train[FEATURES].copy()

    X_test = test[FEATURES].copy()
    X_train, X_valid, y_train, y_valid = model_selection.train_test_split(
        X, y, test_size=0.2, random_state=Config.seed
    )

# === XGB ===

time_limit = 3600 * 3
best_xgb_params = {}

if Config.optimize:
    study = optuna.create_study(direction=objective_direction)
    study.optimize(
        lambda trial: objective_xgb(trial, X_train, X_valid, y_train, y_valid),
        n_trials=Config.n_optuna_trials,
        # timeout=time_limit,  # this or n_trials
    )

if Config.optimize:
    print("Number of finished trials:", len(study.trials))
    print("Best XGB trial parameters:", study.best_trial.params)
    print("Best score:", study.best_value)
    best_xgb_params = study.best_trial.params

## === LGBM ===

time_limit = 3600 * 3
best_lgbm_params = {}

if Config.optimize:
    study = optuna.create_study(direction=objective_direction)
    study.optimize(
        lambda trial: objective_lgbm(trial, X_train, X_valid, y_train, y_valid),
        n_trials=Config.n_optuna_trials,
        # timeout=time_limit,  # this or n_trials
    )

if Config.optimize:
    print("Number of finished trials:", len(study.trials))
    print("Best LGBM trial parameters:", study.best_trial.params)
    print("Best score:", study.best_value)
    best_lgbm_params = study.best_trial.params

## === CatBoost

time_limit = 3600 * 3
# best_cb_params = {}
best_cb_params = {'learning_rate': 0.45743264601999495,
                  'l2_leaf_reg': 41.338946049390074,
                  'bagging_temperature': 0.3472567739474319,
                  'random_strength': 1.7332249677756242, 
                  'depth': 1,
                  'min_data_in_leaf': 6}

if Config.optimize:
    study = optuna.create_study(direction=objective_direction)
    study.optimize(
        lambda trial: objective_cb(trial, X_train, X_valid, y_train, y_valid),
        n_trials=Config.n_optuna_trials,
        # timeout=time_limit,  # this or n_trials
    )

if Config.optimize:
    print("Number of finished trials:", len(study.trials))
    print("Best Cat trial parameters:", study.best_trial.params)
    print("Best score:", study.best_value)
    best_cb_params = study.best_trial.params

[I 2023-03-13 21:59:49,529] A new study created in memory with name: no-name-2b39018f-e8b6-4869-b4ea-7433cde4c206
[I 2023-03-13 21:59:56,684] Trial 0 finished with value: 11.405228378368628 and parameters: {'use_label_encoder': False, 'n_estimators': 1100, 'learning_rate': 0.018837121833560722, 'subsample': 0.69, 'colsample_bytree': 0.4, 'max_depth': 6, 'gamma': 27.0, 'booster': 'gbtree', 'tree_method': 'hist', 'reg_lambda': 0.0069716684272638635, 'reg_alpha': 0.00041064786783973864, 'random_state': 42, 'n_jobs': 4, 'min_child_weight': 205.73557207698073}. Best is trial 0 with value: 11.405228378368628.


Number of boosting rounds: 597


[I 2023-03-13 22:00:03,262] Trial 1 finished with value: 12.272559284733667 and parameters: {'use_label_encoder': False, 'n_estimators': 3200, 'learning_rate': 0.048185011412229105, 'subsample': 0.77, 'colsample_bytree': 0.35, 'max_depth': 20, 'gamma': 7.2, 'booster': 'gbtree', 'tree_method': 'hist', 'reg_lambda': 0.031886075274786777, 'reg_alpha': 0.35996267311765734, 'random_state': 42, 'n_jobs': 4, 'min_child_weight': 0.16536845058458505}. Best is trial 0 with value: 11.405228378368628.


Number of boosting rounds: 78


[I 2023-03-13 22:00:13,424] Trial 2 finished with value: 11.460252776708229 and parameters: {'use_label_encoder': False, 'n_estimators': 1700, 'learning_rate': 0.026106844073122592, 'subsample': 0.88, 'colsample_bytree': 0.11, 'max_depth': 3, 'gamma': 75.4, 'booster': 'gbtree', 'tree_method': 'hist', 'reg_lambda': 6.537742635189102e-06, 'reg_alpha': 0.015045013758341382, 'random_state': 42, 'n_jobs': 4, 'min_child_weight': 206.20782094200592}. Best is trial 0 with value: 11.405228378368628.


...

Early stopping, best iteration is:
[441]	training's l1: 8.33782	valid_1's l1: 8.64057
[LightGBM] [Warning] bagging_fraction is set=0.9752057290133664, subsample=0.97 will be ignored. Current value: bagging_fraction=0.9752057290133664
[LightGBM] [Warning] feature_fraction is set=0.30347326428863886, colsample_bytree=0.19 will be ignored. Current value: feature_fraction=0.30347326428863886
[LightGBM] [Warning] bagging_freq is set=14, subsample_freq=0 will be ignored. Current value: bagging_freq=14
Training until validation scores don't improve for 500 rounds
[500]	training's l1: 8.44326	training's rmse: 11.1028	valid_1's l1: 8.78592	valid_1's rmse: 11.4411


[I 2023-03-13 22:19:35,118] Trial 99 finished with value: 11.430763958272097 and parameters: {'objective': 'rmse', 'n_estimators': 3831, 'reg_alpha': 1.6133843616360049e-06, 'reg_lambda': 0.0011009106351766043, 'colsample_bytree': 0.19, 'num_leaves': 728, 'feature_fraction': 0.30347326428863886, 'bagging_fraction': 0.9752057290133664, 'bagging_freq': 14, 'min_child_samples': 231, 'subsample': 0.97, 'learning_rate': 0.019790905461139704, 'max_depth': 59, 'random_state': 42, 'n_jobs': 4}. Best is trial 94 with value: 11.392108217508795.
[I 2023-03-13 22:19:35,155] A new study created in memory with name: no-name-67e7dcfa-3dc3-4b31-aeb0-64dda5681902


Early stopping, best iteration is:
[418]	training's l1: 8.49712	training's rmse: 11.1707	valid_1's l1: 8.77876	valid_1's rmse: 11.4308
Number of finished trials: 100
Best LGBM trial parameters: {'objective': 'rmse', 'n_estimators': 4185, 'reg_alpha': 3.594090840805944e-07, 'reg_lambda': 0.00046888907983248074, 'colsample_bytree': 0.12000000000000001, 'num_leaves': 782, 'feature_fraction': 0.37993960583931685, 'bagging_fraction': 0.9784794083554482, 'bagging_freq': 14, 'min_child_samples': 292, 'subsample': 0.95, 'learning_rate': 0.022325335718188847, 'max_depth': 63, 'random_state': 42, 'n_jobs': 4}
Best score: 11.392108217508795


[I 2023-03-13 22:19:36,447] Trial 0 finished with value: 11.535532519164486 and parameters: {'learning_rate': 0.29954086137826197, 'l2_leaf_reg': 3.6010616323022644, 'bagging_temperature': 0.27493284862706713, 'random_strength': 1.8267911924287876, 'depth': 9, 'min_data_in_leaf': 227}. Best is trial 0 with value: 11.535532519164486.
[I 2023-03-13 22:19:36,638] Trial 1 finished with value: 11.44497748705865 and parameters: {'learning_rate': 0.1334986507283711, 'l2_leaf_reg': 1.8655950246089918, 'bagging_temperature': 0.133155896284047, 'random_strength': 1.2129162065181514, 'depth': 3, 'min_data_in_leaf': 192}. Best is trial 1 with value: 11.44497748705865.
[I 2023-03-13 22:19:36,833] Trial 2 finished with value: 11.453521594807205 and parameters: {'learning_rate': 0.2655139196028855, 'l2_leaf_reg': 85.10733454111343, 'bagging_temperature': 2.8665626404809603, 'random_strength': 1.9522451608921305, 'depth': 3, 'min_data_in_leaf': 60}. Best is trial 1 with value: 11.44497748705865.
[I 20

Number of finished trials: 100
Best Cat trial parameters: {'learning_rate': 0.12265251412174061, 'l2_leaf_reg': 6.246670134102711, 'bagging_temperature': 0.37821180665912224, 'random_strength': 1.734149070693527, 'depth': 5, 'min_data_in_leaf': 148}
Best score: 11.387698041130266
CPU times: user 1h 43s, sys: 3min 48s, total: 1h 4min 32s
Wall time: 20min 25s


<div style="background-color:rgba(177, 156, 217, 0.6);border-radius:5px;display:fill"><h1 style="text-align: center;padding: 12px 0px 12px 0px;">Train Models with Cross Validation</h1>
</div>

In [26]:
def show_tree_model_fi(model, features:List[str]) -> None:
    print("\n=== Model Feature Importance ===")
    for i in model.feature_importances_.argsort()[::-1]:
        print(features[i], model.feature_importances_[i]/model.feature_importances_.sum())

def save_oof_predictions(model_name:str, final_valid_predictions, oof:pd.DataFrame) -> pd.DataFrame:
    final_valid_predictions_df = process_valid_predictions(
        final_valid_predictions, ID, model_name
    )
    display(final_valid_predictions_df.head())
    oof[f"pred_{model_name}"] = final_valid_predictions_df[f"pred_{model_name}"]

    return oof

def save_test_predictions(model_name:str, final_test_predictions, submission_df:pd.DataFrame, result_field:str=TARGET) -> None:
    result = merge_test_predictions(final_test_predictions, Config.calc_probability)
    # result[:20]
    submission_df[f"target_{model_name}"] = result #.astype(int)
    #     submission_df.head(10)
    ss = submission_df[[ID, f"target_{model_name}"]].copy().reset_index(drop=True)
    ss.rename(columns={f"target_{model_name}": result_field}, inplace=True)
    ss.to_csv(
        f"submission_{model_name}.csv", index=False
    )  # Can submit the individual model
    print("=== Target Value Counts ===")
#     display(ss[TARGET].value_counts())
    ss.head(10)

def process_valid_predictions(final_valid_predictions, train_id, model_name:str) -> pd.DataFrame:
    model = f"pred_{model_name}"
    final_valid_predictions_df = pd.DataFrame.from_dict(
        final_valid_predictions, orient="index"
    ).reset_index()
    final_valid_predictions_df.columns = [train_id, model]
    final_valid_predictions_df.set_index(train_id, inplace=True)
    final_valid_predictions_df.sort_index(inplace=True)
    final_valid_predictions_df.to_csv(f"train_pred_{model_name}.csv", index=True)

    return final_valid_predictions_df

def add_score(score_df:pd.DataFrame, model_name:str, score:float, std:float):
    dict1 = {"Model": model_name, "Score": cv_score, "StdDev": std_dev}
    score_df = score_df.append(dict1, ignore_index=True)
    return score_df

In [27]:
def train_cv_model(
    df:pd.DataFrame,
    test:pd.DataFrame,
    get_model_fn,
    FEATURES:List[str],
    TARGET:str,
    calc_probability:bool,
    rowid,
    params,
    n_folds:int=5,
    seed:int=42,
):

    final_test_predictions = []
    final_valid_predictions = {}
    fold_scores = []  # Scores of Validation Set
    feature_importance_lst = []

    test = test[FEATURES].copy()

    for fold in range(n_folds):
        print(10 * "=", f"Fold {fold+1}/{n_folds}", 10 * "=")

        start_time = time.time()

        xtrain = df[df.fold != fold].reset_index(
            drop=True
        )  # Everything not in validation fold
        xvalid = df[df.fold == fold].reset_index(drop=True)
        xtest = test.copy()

        valid_ids = xvalid.id.values.tolist()  # Id's of everything in validation fold

        ytrain = xtrain[TARGET]
        yvalid = xvalid[TARGET]

        xtrain = xtrain[FEATURES]
        xvalid = xvalid[FEATURES]

        scaler = preprocessing.StandardScaler()
#         scaler = preprocessing.MinMaxScaler()
        xtrain = scaler.fit(xtrain).transform(xtrain)
        xvalid = scaler.transform(xvalid)
        xtest = scaler.transform(xtest)

        model = get_model_fn # ()

        model.fit(
            xtrain,
            ytrain,
        )
        if calc_probability:
            preds_valid = model.predict_proba(xvalid)[:, 1]
            test_preds = model.predict_proba(xtest)[:, 1]
        else:
            preds_valid = model.predict(xvalid)
            test_preds = model.predict(xtest)

        preds_valid_class = model.predict(xvalid)
        
        final_test_predictions.append(test_preds)
        final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))

#         fold_score = metrics.accuracy_score(yvalid, preds_valid_class)  # Validation Set Score
        fold_score = metrics.mean_absolute_error(
            yvalid, preds_valid
        ) 
#         fold_score = metrics.roc_auc_score(yvalid.values, preds_valid)  # Validation Set Score
#         show_classification_scores(yvalid.values, preds_valid_class)

#         fold_score = metrics.roc_auc_score(yvalid, preds_valid)  # Validation Set Score
#         fold_score = metrics.mean_squared_error(yvalid, preds_valid, squared=False)
        fold_scores.append(fold_score)
        #         importance_list.append(model.coef_.ravel())

        fi = []
        # Feature importance
#         fi = pd.DataFrame(
#             index=FEATURES,
#             data=model.coef_.ravel(),
#             columns=[f"{fold}_importance"],
#         )
        
        feature_importance_lst.append(fi)

        run_time = time.time() - start_time

        print(f"fold: {fold+1}, Score: {fold_score}, Run Time: {run_time:.2f}")

    return (
        model,
        feature_importance_lst,
        fold_scores,
        final_valid_predictions,
        final_test_predictions,
    )


def train_xgb_model(
    df:pd.DataFrame,
    test:pd.DataFrame,
    get_model_fn,
    FEATURES:List[str],
    TARGET:str,
    calc_probability:bool,
    rowid:str,
    params,
    n_folds:int=5,
    seed:int=42,
):

    print(params)
    final_test_predictions = []
    final_valid_predictions = {}
    fold_scores = []  # Scores of Validation Set
    feature_importance_lst = []

    test = test[FEATURES].copy()

    for fold in range(n_folds):
        print(10 * "=", f"Fold {fold+1}/{n_folds}", 10 * "=")

        start_time = time.time()

        xtrain = df[df.fold != fold].reset_index(
            drop=True
        )  # Everything not in validation fold
        xvalid = df[df.fold == fold].reset_index(drop=True)
        xtest = test.copy()

        valid_ids = xvalid.id.values.tolist()  # Id's of everything in validation fold

        ytrain = xtrain[TARGET]
        yvalid = xvalid[TARGET]

        xtrain = xtrain[FEATURES]
        xvalid = xvalid[FEATURES]

        model = get_model_fn # (params)

        model.fit(
            xtrain,
            ytrain,
            eval_set=[(xvalid, yvalid)],
            #             eval_metric="acc",  # auc
            verbose=0,
            early_stopping_rounds=500,
            #             early_stopping_rounds=3000,
            #             callbacks=[
            #                 xgb.log_evaluation(0),
            #                 xgb.early_stopping(500, False, True),
            #             ],
        )

        if calc_probability:
            preds_valid = model.predict_proba(xvalid)[:, 1]
            test_preds = model.predict_proba(xtest)[:, 1]
        else:
            preds_valid = model.predict(xvalid)
            test_preds = model.predict(xtest)

        preds_valid_class = model.predict(xvalid)
        
        final_test_predictions.append(test_preds)
        if Config.debug:
            print(f"GT Type: {type(yvalid.values)}")
            print(f"Preds Type: {type(preds_valid_class)}")
            print(f"         GT:{yvalid.values[:20]}")
            print(f"Preds Class:{preds_valid_class[:20]}")
            print(f"Preds Prob:{preds_valid[:20]}")
        final_valid_predictions.update(dict(zip(valid_ids, preds_valid_class)))

#         fold_score = metrics.cohen_kappa_score(yvalid,  preds_valid_class, weights = "quadratic")
#         fold_score = metrics.roc_auc_score(yvalid.values, preds_valid)  # Validation Set Score
#         show_classification_scores(yvalid.values, preds_valid_class)
        fold_score = metrics.mean_absolute_error(
            yvalid, preds_valid
        )  # Validation Set Score
#         fold_score = metrics.mean_squared_error(yvalid, preds_valid, squared=False)
        fold_scores.append(fold_score)

        # Feature importance
        fi = pd.DataFrame(
            index=FEATURES,
            data=model.feature_importances_,
            columns=[f"{fold}_importance"],
        )
        feature_importance_lst.append(fi)

        run_time = time.time() - start_time

        print(f"fold: {fold+1}, Score: {fold_score}, Run Time: {run_time:.2f}")

    return (
        model,
        feature_importance_lst,
        fold_scores,
        final_valid_predictions,
        final_test_predictions,
    )        

In [28]:
def run_linear_model(train:pd.DataFrame, test:pd.DataFrame, model_dict, model_name:str, features:List[str], oof:pd.DataFrame) -> (float, float, pd.DataFrame):
    (
        model,
        feature_importance_lst,
        fold_scores,
        final_valid_predictions,
        final_test_predictions,
    ) = train_cv_model(
        train,
        test,
        model_dict[model_name],
        features,
        TARGET,
        False, #Config.calc_probability,
        ID,
        {},
        Config.N_FOLDS,
        Config.seed,
    )

    cv_score, std_dev = show_fold_scores(fold_scores)

    oof = save_oof_predictions(model_name, final_valid_predictions, oof)
    oof.head()
    save_test_predictions(model_name, final_test_predictions, sample_submission, TARGET)

    return cv_score, std_dev, oof


def run_tree_model(train:pd.DataFrame, test:pd.DataFrame, model_dict, model_name:str, features:List[str], params, oof:pd.DataFrame) -> (float, float, pd.DataFrame):
    (
        model,
        feature_importance_lst,
        fold_scores,
        final_valid_predictions,
        final_test_predictions,
    ) = train_xgb_model(
        train,
        test,
        model_dict[model_name],
        features,
        TARGET,
        Config.calc_probability,
        ID,
        params,
        Config.N_FOLDS,
        Config.seed,
    )

    cv_score, std_dev = show_fold_scores(fold_scores)
    show_tree_model_fi(model, features)

    oof = save_oof_predictions(model_name, final_valid_predictions, oof)
    oof.head()
    save_test_predictions(model_name, final_test_predictions, sample_submission, TARGET)

    return cv_score, std_dev, oof

In [29]:
%%time

def run_models4features(train:pd.DataFrame, test:pd.DataFrame, model_dict, model_lst:List[str], target:str, feature_lst:List[str], all_cv_scores:pd.DataFrame, linear_models:bool=True) -> pd.DataFrame:

    oof = train[[ID, target, "fold"]].copy().reset_index(drop=True).copy()
    oof.set_index(ID, inplace=True)

    for idx, m in enumerate(model_lst):
        model = model_lst[idx]
        start_time = time.time()

        print(f"Model={model}")

        params = {}
        if linear_models:
                cv_score, std_dev, oof = run_linear_model(train, test, model_dict, model, feature_lst, oof)

        else:
            cv_score, std_dev, oof = run_tree_model(train, test, model_dict, model, feature_lst, params, oof)

        run_time = time.time() - start_time

        score_dict = {"Model": model, "Score": cv_score, "StdDev": std_dev, "RunTime": run_time, "n_estimators": Config.N_ESTIMATORS, "n_folds": Config.N_FOLDS, "comments": ""}
        all_cv_scores = all_cv_scores.append(score_dict, ignore_index=True)
        print(f"Model Run Time: {run_time:.2f}")

    return all_cv_scores




CPU times: user 15 µs, sys: 1e+03 ns, total: 16 µs
Wall time: 20.3 µs


In [30]:
lgbm_params = {'n_estimators': Config.N_ESTIMATORS,
                 'num_rounds': 404,
                 'learning_rate': 0.19,
                 'num_leaves': 17,
                 'max_depth': 8,
                 'min_data_in_leaf': 36,
                 'lambda_l1': 0.96,
                 'lambda_l2': 0.01,
                 'min_gain_to_split': 11.32,
                 'bagging_fraction': 0.6,
                 'feature_fraction': 0.9}


lgbm_params3 = {
    "n_estimators": Config.N_ESTIMATORS,
    'max_depth': 9,
    'learning_rate': 0.01,
    'min_data_in_leaf': 36, 
    'num_leaves': 100, 
    'feature_fraction': 0.8, 
    'bagging_fraction': 0.89, 
    'bagging_freq': 5, 
    'lambda_l2': 28,
    
    'seed': Config.seed,
    'objective': 'regression',
#     'boosting_type': 'gbdt',
#     'device': 'gpu', 
#     'gpu_platform_id': 0,
#     'gpu_device_id': 0,
    'n_jobs': -1,
    'metric': 'rmse',
    'verbose': -1
}
    
lgbm_params = gpu_ify_lgbm(lgbm_params)

In [31]:
xgb_params = {
    "n_estimators": Config.N_ESTIMATORS,  # 10_000,
    "max_depth": 10,  # 10
    "objective": "reg:squarederror",
    #     "enable_categorical": True,  # Only works with gpu_hist
    #     "eval_metric": "mae",
    #     "metric": "mae",
    #     "enable_categorical": True,
    "n_jobs": 8,  # 4
    "seed": Config.seed,
    "tree_method": "hist",
    #         "gpu_id": 0,
    "subsample": 0.9,  # 0.7
    "colsample_bytree": 0.7,
    "use_label_encoder": False,
    "learning_rate": 0.05,  # 0.01
}

xgb_params3 = {
    'n_estimators': Config.N_ESTIMATORS,
    'learning_rate': 0.05,
    'max_depth': 10,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'objective': 'reg:squarederror'
}

if Config.gpu:
    xgb_params["tree_method"] = "gpu_hist"
else:
    xgb_params["tree_method"] = "hist"

In [32]:
cb_params = {
    #     "learning_rate": 0.3277295792305584,
    "learning_rate": 0.05,
    "l2_leaf_reg": 3.1572972266001518,
    "bagging_temperature": 0.6799604234141348,
    "random_strength": 1.99590400593318,
    "depth": 10,
    "min_data_in_leaf": 93,
    # "iterations": 100,  # 10000
    "n_estimators": Config.N_ESTIMATORS,  # 10000
    "use_best_model": True,
    #     "task_type": "GPU",
    "random_seed": Config.seed,
}

cb_params = gpu_ify_cb(cb_params)

In [33]:
model_estimator_dict = {
    "xgb2": xgb.XGBRegressor(**xgb_params),
    "xgb_best_params": xgb.XGBRegressor(**best_xgb_params),
    "xgb3": xgb.XGBRegressor(**xgb_params3),


    "lgbm1": lgb.LGBMRegressor(**lgbm_params),

    "cat1": cb.CatBoostRegressor(),
    "cat2": cb.CatBoostRegressor(**cb_params),
    "cat_best_params": cb.CatBoostRegressor(**best_cb_params),

    "xgb1": xgb.XGBRegressor(),
    "lgbm0": lgb.LGBMRegressor(),
    "lgbm3": lgb.LGBMRegressor(lgbm_params3),
    "lgbm2": lgb.LGBMRegressor(
        learning_rate=0.05,
        max_depth=15,
        num_leaves=11,
        feature_fraction=0.3,
        subsample=0.1,
        n_jobs=-1,
    ),
    "lgbm3": lgb.LGBMRegressor(**lgbm_params),
    "lgbm_best_params": lgb.LGBMRegressor(**best_lgbm_params),


    "lin_reg": linear_model.LinearRegression(),
    "lasso": linear_model.Lasso(),
    "ridge": linear_model.Ridge(max_iter=7000),
    "ridge_25": linear_model.Ridge(fit_intercept=True, solver='auto', alpha=0.25, max_iter=7000),
    "ridge_50": linear_model.Ridge(fit_intercept=True, solver='auto', alpha=0.5, max_iter=7000),
}

In [34]:
all_cv_scores = pd.DataFrame(
    {
        "Model": pd.Series(dtype="str"),
        "Score": pd.Series(dtype="float"),
        "StdDev": pd.Series(dtype="float"),
        "RunTime": pd.Series(dtype="float"),
        "n_estimators": pd.Series(dtype="int"),
        "n_folds": pd.Series(dtype="int"),
        "comments": pd.Series(dtype="str"),
    }
)



## Tree Models

In [35]:
%%time

# model_lst = ["xgb3","xgb_best_params", "lgbm_best_params", "cat_best_params", "xgb1", "xgb2", "lgbm1", "lgbm2", "cat1", "cat2"]
# model_lst = = []

def run_tree_models(X_tr, test, n_folds, model_lst, all_cv_scores):
        all_cv_scores = run_models4features(X_tr, test, model_estimator_dict, model_lst, TARGET, FEATURES, all_cv_scores, linear_models=False)    

        all_cv_scores.sort_values(by=["Score"], ascending=False)
        return all_cv_scores

CPU times: user 6 µs, sys: 1e+03 ns, total: 7 µs
Wall time: 9.78 µs


## Linear Models

In [36]:
def run_linear_models(X_tr, test, n_folds, model_lst, all_cv_scores):
    for training in [train, train]:

    #     all_cv_scores = run_models4features(train, test, model_estimator_dict, model_lst, TARGET, FEATURES, all_cv_scores, linear_models=False)    
        all_cv_scores = run_models4features(X_tr, test, model_estimator_dict, model_lst, TARGET, FEATURES, all_cv_scores, linear_models=True)    
        return all_cv_scores  

In [37]:
%%time


tree_model_lst = ["xgb_best_params", "lgbm_best_params", "cat_best_params","xgb3", "xgb1", "xgb2", "lgbm0", "lgbm1", "lgbm2", "lgbm3", "cat1", "cat2"]
linear_model_lst = ["lin_reg", "lasso", "ridge", "ridge_25", "ridge_50"]
linear_model_lst = ["lasso", "ridge",  "ridge_50"]

# Config.N_FOLDS = 10

for training in [train]:
    X_tr = create_folds(train, Config.N_FOLDS)
    
    oof = train[[ID, TARGET, "fold"]].copy().reset_index(drop=True).copy()
    oof.set_index(ID, inplace=True)
    oof.head()

    all_cv_scores = run_tree_models(X_tr, test, Config.N_FOLDS, tree_model_lst, all_cv_scores)
    all_cv_scores = run_linear_models(X_tr, test, Config.N_FOLDS, linear_model_lst, all_cv_scores)
#     Config.N_FOLDS = 5



n_folds=10, seed=42
Model=xgb_best_params
{}
========== Fold 1/10 ==========
fold: 1, Score: 9.110026021744153, Run Time: 4.86
========== Fold 2/10 ==========
fold: 2, Score: 8.291214056785062, Run Time: 5.78
========== Fold 3/10 ==========
fold: 3, Score: 8.766407887950447, Run Time: 4.82
========== Fold 4/10 ==========
fold: 4, Score: 8.452901015311296, Run Time: 3.75
========== Fold 5/10 ==========
fold: 5, Score: 8.463303337215638, Run Time: 4.91
========== Fold 6/10 ==========
fold: 6, Score: 9.347290151607917, Run Time: 4.90
========== Fold 7/10 ==========
fold: 7, Score: 9.303803450069072, Run Time: 4.88
========== Fold 8/10 ==========
fold: 8, Score: 8.715843988777504, Run Time: 4.90
========== Fold 9/10 ==========
fold: 9, Score: 9.020923654903335, Run Time: 5.88
========== Fold 10/10 ==========
fold: 10, Score: 9.099061634840899, Run Time: 4.78
Scores -> Adjusted: 8.50267049 , mean: 8.85707752, std: 0.35440703

=== Model Feature Importance ===
AgeInDays 0.41759464
Age_Water 0

,pred_xgb_best_params
id,
0.0,22.13027
1.0,31.40763
2.0,39.08797
3.0,45.28404
4.0,44.85145


Mode
=== Target Value Counts ===
Model Run Time: 49.64
Model=lgbm_best_params
{}
========== Fold 1/10 ==========
[LightGBM] [Warning] bagging_fraction is set=0.9784794083554482, subsample=0.95 will be ignored. Current value: bagging_fraction=0.9784794083554482
[LightGBM] [Warning] feature_fraction is set=0.37993960583931685, colsample_bytree=0.12000000000000001 will be ignored. Current value: feature_fraction=0.37993960583931685
[LightGBM] [Warning] bagging_freq is set=14, subsample_freq=0 will be ignored. Current value: bagging_freq=14
fold: 1, Score: 9.137737515657493, Run Time: 4.26
========== Fold 2/10 ==========
[LightGBM] [Warning] bagging_fraction is set=0.9784794083554482, subsample=0.95 will be ignored. Current value: bagging_fraction=0.9784794083554482
[LightGBM] [Warning] feature_fraction is set=0.37993960583931685, colsample_bytree=0.12000000000000001 will be ignored. Current value: feature_fraction=0.37993960583931685
[LightGBM] [Warning] bagging_freq is set=14, subsample_

,pred_lgbm_best_params
id,
0.0,21.71985
1.0,32.35834
2.0,38.69073
3.0,45.54748
4.0,43.94822


Mode
=== Target Value Counts ===
Model Run Time: 36.61
Model=cat_best_params
{}
========== Fold 1/10 ==========
fold: 1, Score: 9.137346137282188, Run Time: 1.77
========== Fold 2/10 ==========
fold: 2, Score: 8.251997076765509, Run Time: 1.34
========== Fold 3/10 ==========
fold: 3, Score: 8.775463077033951, Run Time: 1.29
========== Fold 4/10 ==========
fold: 4, Score: 8.5808709905468, Run Time: 1.28
========== Fold 5/10 ==========
fold: 5, Score: 8.597994221705292, Run Time: 1.29
========== Fold 6/10 ==========
fold: 6, Score: 9.282715331759244, Run Time: 1.60
========== Fold 7/10 ==========
fold: 7, Score: 9.380404708122205, Run Time: 1.31
========== Fold 8/10 ==========
fold: 8, Score: 8.644459589120542, Run Time: 1.89
========== Fold 9/10 ==========
fold: 9, Score: 9.094349377488042, Run Time: 1.63
========== Fold 10/10 ==========
fold: 10, Score: 9.017672589984185, Run Time: 1.51
Scores -> Adjusted: 8.53378149 , mean: 8.87632731, std: 0.34254582

=== Model Feature Importance ===

...

Mode
=== Target Value Counts ===
Model Run Time: 0.96
Model=ridge
========== Fold 1/10 ==========
fold: 1, Score: 10.955055673177444, Run Time: 0.05
========== Fold 2/10 ==========
fold: 2, Score: 10.580603704233269, Run Time: 0.07
========== Fold 3/10 ==========
fold: 3, Score: 10.599294481622278, Run Time: 0.06
========== Fold 4/10 ==========
fold: 4, Score: 10.6823862246851, Run Time: 0.07
========== Fold 5/10 ==========
fold: 5, Score: 11.308475896614617, Run Time: 0.07
========== Fold 6/10 ==========
fold: 6, Score: 11.464555419835758, Run Time: 0.07
========== Fold 7/10 ==========
fold: 7, Score: 11.287080314230932, Run Time: 0.06
========== Fold 8/10 ==========
fold: 8, Score: 10.818972107388642, Run Time: 0.07
========== Fold 9/10 ==========
fold: 9, Score: 11.013553224034151, Run Time: 0.09
========== Fold 10/10 ==========
fold: 10, Score: 10.495787010645362, Run Time: 0.06
Scores -> Adjusted: 10.59548483 , mean: 10.92057641, std: 0.32509158


,pred_ridge
id,
0.0,31.45475
1.0,33.08502
2.0,32.25309
3.0,46.85241
4.0,36.95828


Mode
=== Target Value Counts ===
Model Run Time: 0.91
Model=ridge_50
========== Fold 1/10 ==========
fold: 1, Score: 10.956729386959054, Run Time: 0.04
========== Fold 2/10 ==========
fold: 2, Score: 10.579335667644964, Run Time: 0.07
========== Fold 3/10 ==========
fold: 3, Score: 10.597165702491926, Run Time: 0.06
========== Fold 4/10 ==========
fold: 4, Score: 10.68111824374857, Run Time: 0.07
========== Fold 5/10 ==========
fold: 5, Score: 11.308317399872614, Run Time: 0.06
========== Fold 6/10 ==========
fold: 6, Score: 11.464085803956248, Run Time: 0.06
========== Fold 7/10 ==========
fold: 7, Score: 11.286086533935098, Run Time: 0.07
========== Fold 8/10 ==========
fold: 8, Score: 10.81818550973201, Run Time: 0.09
========== Fold 9/10 ==========
fold: 9, Score: 11.012330868375487, Run Time: 0.06
========== Fold 10/10 ==========
fold: 10, Score: 10.492879703741936, Run Time: 0.08
Scores -> Adjusted: 10.59391681 , mean: 10.91962348, std: 0.32570667


,pred_ridge_50
id,
0.0,31.42529
1.0,33.09282
2.0,32.25117
3.0,46.66750
4.0,36.96721


Mode
=== Target Value Counts ===
Model Run Time: 0.91
CPU times: user 15min 43s, sys: 1min 21s, total: 17min 5s
Wall time: 7min 26s


In [38]:
sample_submission.head(20)

,id,Strength,target_xgb_best_params,target_lgbm_best_params,target_cat_best_params,target_xgb3,target_xgb1,target_xgb2,target_lgbm0,target_lgbm1,target_lgbm2,target_lgbm3,target_cat1,target_cat2,target_lasso,target_ridge,target_ridge_50
0,5407,35.452,46.34083,47.49565,46.70121,42.98643,44.49270,44.64078,45.78327,45.94662,45.91039,45.94662,47.67475,44.18114,33.92659,34.54784,34.55878
1,5408,35.452,18.80961,18.95663,17.85010,19.66376,18.57957,17.55502,19.48269,18.78587,20.03686,18.78587,17.88198,18.09641,28.70706,26.50845,26.50955
2,5409,35.452,31.25490,31.62801,31.19775,30.16411,30.35761,29.61822,31.73340,31.61498,32.80145,31.61498,31.42835,32.43166,28.56778,25.44775,25.44451
3,5410,35.452,46.62706,46.78770,46.71389,44.31639,45.07124,44.99257,45.25928,45.40394,44.13508,45.40394,46.61178,45.72016,38.60147,39.97510,39.99466
4,5411,35.452,27.73982,27.66084,20.64283,16.12865,17.16118,16.71099,21.80552,25.05543,27.44550,25.05543,20.80786,20.47853,31.16459,28.15894,28.15685
5,5412,35.452,40.11304,41.86407,39.34982,36.90488,38.88086,32.54675,41.39774,41.27538,40.61248,41.27538,39.54350,40.11158,34.34905,35.52206,35.53241
6,5413,35.452,34.28734,34.81260,34.56696,30.46301,31.39770,29.74210,29.63408,33.36673,33.76486,33.36673,33.73834,33.30835,29.26426,26.63645,26.62976
7,5414,35.452,20.74249,20.87285,21.31171,19.46043,19.11687,19.55410,21.13161,20.88858,22.02147,20.88858,21.42524,20.88549,30.10257,31.87054,31.86845
8,5415,35.452,47.29796,46.63235,46.54344,38.31536,34.77943,42.17943,43.38155,45.69631,43.39129,45.69631,45.60498,39.65233,36.79924,39.14495,39.11224
9,5416,35.452,37.58529,38.14060,38.34467,31.51257,33.17751,34.58575,38.68226,36.55959,37.67738,36.55959,37.85778,37.46378,32.83793,31.22644,31.22726


<div style="background-color:rgba(177, 156, 217, 0.6);border-radius:5px;display:fill"><h1 style="text-align: center;padding: 12px 0px 12px 0px;">Blend Models</h1>
</div>

In [39]:
all_blend_scores = pd.DataFrame(
    {
        "Model": pd.Series(dtype="str"),
        "Score": pd.Series(dtype="float"),
        "StdDev": pd.Series(dtype="float"),
    }
)

In [40]:
model_lst = ["xgb3", "xgb2", "cat1", "lgbm0", "lgbm1","lgbm2", "cat_best_params", "lgbm_best_params"]

In [41]:
len(model_lst)

8

In [42]:
target_names = [f"target_{model}" for model in model_lst]
target_names

['target_xgb3',
 'target_xgb2',
 'target_cat1',
 'target_lgbm0',
 'target_lgbm1',
 'target_lgbm2',
 'target_cat_best_params',
 'target_lgbm_best_params']

In [43]:
sample_submission[TARGET] = sample_submission[target_names].sum(axis=1) / len(model_lst)

In [44]:
sample_submission[[ID, TARGET]].to_csv("submission_equal_wt_avg.csv", index=False)
sample_submission[[ID, TARGET]].tail(8)

,id,Strength
3597,9004,18.07665
3598,9005,39.24116
3599,9006,16.73651
3600,9007,30.46711
3601,9008,30.99574
3602,9009,40.64909
3603,9010,33.05177
3604,9011,21.83249


In [45]:
sample_submission[TARGET] = (
#     (sample_submission["target_xgb_bp"] * 2 )
#     + (sample_submission["target_lgbm_bp"]  )
    (sample_submission["target_cat_best_params"] * 3 )
    + (sample_submission["target_lgbm1"])
#     + (sample_submission["target_lgbm2"])    
#     + (sample_submission["target_lgbm2"])
    + (sample_submission["target_cat1"] )
    + (sample_submission["target_cat2"] )    
#     + (sample_submission["target_cat_bp"] )
#     + (sample_submission["target_svc"] )
#     + (sample_submission["target_log_reg3"] )
#     + (sample_submission["target_cat2"] )
)/6

# sample_submission[TARGET] = sample_submission[TARGET].astype(int)

In [46]:
sample_submission[[ID, TARGET]].to_csv("submission_wt_avg.csv", index=False)
sample_submission[[ID, TARGET]].tail(8)

,id,Strength
3597,9004,17.69381
3598,9005,39.72191
3599,9006,15.45613
3600,9007,30.29333
3601,9008,32.52441
3602,9009,39.37280
3603,9010,33.54932
3604,9011,20.69976


In [47]:
all_cv_scores.sort_values(by=["Score"], ascending=False)

,Model,Score,StdDev,RunTime,n_estimators,n_folds,comments
12,lasso,11.33242,0.29682,0.95861,500,10,
13,ridge,10.92058,0.32509,0.90770,500,10,
14,ridge_50,10.91962,0.32571,0.90761,500,10,
5,xgb2,9.06998,0.33052,83.73373,500,10,
3,xgb3,9.05538,0.34873,95.93039,500,10,
8,lgbm2,8.98837,0.30146,3.42690,500,10,
4,xgb1,8.96635,0.35175,12.91841,500,10,
6,lgbm0,8.94331,0.29525,8.54554,500,10,
11,cat2,8.91413,0.36479,99.90426,500,10,
7,lgbm1,8.90239,0.33008,8.78842,500,10,


<div style="background-color:rgba(177, 156, 217, 0.6);border-radius:5px;display:fill"><h1 style="text-align: center;padding: 12px 0px 12px 0px;">Level 1 Stack Models</h1>
</div>

In [48]:
## TODO: Generate these dictionaries from model names

train_oof_dict = {
    "train_pred_cat1": "train_pred_cat1.csv",
    "train_pred_cat2": "train_pred_cat2.csv",
    "train_pred_lgbm1": "train_pred_lgbm1.csv",    
    "train_pred_lgbm2": "train_pred_lgbm2.csv",    
    "train_pred_xgb1": "train_pred_xgb1.csv"
}

test_pred_dict = {
    "submission_cat1": "submission_cat1.csv",
    "submission_cat2": "submission_cat2.csv",
    "submission_lgbm1": "submission_lgbm1.csv",
    "submission_lgbm2": "submission_lgbm2.csv",
    "submission_xgb1": "submission_xgb1.csv",
}

In [49]:
def blend_results(train_oof_dict, test_pred_dict):
    oof_df = pd.DataFrame()
    test_preds_df = pd.DataFrame()

    for name, train_oof_fname in train_oof_dict.items():
        fname = "../working/" + train_oof_fname
        print(f"Processing {name}, {train_oof_fname}")
        df = pd.read_csv(fname)
        print(df.head())
#         print(df.iloc[:,1])
        preds = pd.Series(df.iloc[:,1], name=name)
#         print(preds[:5])
        oof_df = pd.concat([oof_df, preds], axis=1)
    #     oof_df = pd.concat([oof_df, pd.Series(np.load(TRAIN_PATH / train_oof), name=name)], axis=1)

    for name, test_pred_fname in test_pred_dict.items():
        fname = "../working/" + test_pred_fname
        print(f"{name}, {test_pred_fname}")
        df = pd.read_csv(fname)
        print(df.head())
        preds = pd.Series(df.iloc[:,1], name=name)
        test_preds_df = pd.concat([test_preds_df, preds], axis=1)

    print("=== oof ===")
    print(oof_df.head())
    print("=== test_preds ===")
    print(test_preds_df.head())
    return oof_df, test_preds_df
    
# (oof_df, preds_df) = blend_results(train_oof_dict, test_pred_dict)    

In [50]:
def load_oof_results(train_oof_dict, test_pred_dict):
    oof_df = pd.DataFrame()
    test_preds_df = pd.DataFrame()

    for name, train_oof_fname in train_oof_dict.items():
        fname = "../working/" + train_oof_fname
        print(f"Processing {name}, {train_oof_fname}")
        df = pd.read_csv(fname)
        print(df.head())
#         print(df.iloc[:,1])
        preds = pd.Series(df.iloc[:,1], name=name)
#         print(preds[:5])
        oof_df = pd.concat([oof_df, preds], axis=1)
    #     oof_df = pd.concat([oof_df, pd.Series(np.load(TRAIN_PATH / train_oof), name=name)], axis=1)

    for name, test_pred_fname in test_pred_dict.items():
        fname = "../working/" + test_pred_fname
        print(f"{name}, {test_pred_fname}")
        df = pd.read_csv(fname)
        print(df.head())
        preds = pd.Series(df.iloc[:,1], name=name)
        test_preds_df = pd.concat([test_preds_df, preds], axis=1)

    print("=== oof ===")
    print(oof_df.head())
    print("=== test_preds ===")
    print(test_preds_df.head())
    return oof_df, test_preds_df
    
(oof_df, preds_df) = load_oof_results(train_oof_dict, test_pred_dict) 

Processing train_pred_cat1, train_pred_cat1.csv
    id  pred_cat1
0  0.0   21.82535
1  1.0   32.08575
2  2.0   37.24590
3  3.0   45.30529
4  4.0   45.21940
Processing train_pred_cat2, train_pred_cat2.csv
    id  pred_cat2
0  0.0   22.50112
1  1.0   31.04042
2  2.0   38.44881
3  3.0   44.46108
4  4.0   44.77414
Processing train_pred_lgbm1, train_pred_lgbm1.csv
    id  pred_lgbm1
0  0.0    21.94135
1  1.0    33.88222
2  2.0    40.30427
3  3.0    43.66965
4  4.0    44.60095
Processing train_pred_lgbm2, train_pred_lgbm2.csv
    id  pred_lgbm2
0  0.0    22.80341
1  1.0    33.58557
2  2.0    36.28794
3  3.0    44.86389
4  4.0    41.66360
Processing train_pred_xgb1, train_pred_xgb1.csv
    id  pred_xgb1
0  0.0   25.17577
1  1.0   28.23302
2  2.0   37.91516
3  3.0   44.94661
4  4.0   44.63388
submission_cat1, submission_cat1.csv
     id  Strength
0  5407  47.67475
1  5408  17.88198
2  5409  31.42835
3  5410  46.61178
4  5411  20.80786
submission_cat2, submission_cat2.csv
     id  Strength
0  5

In [51]:
oof_df.head()

,train_pred_cat1,train_pred_cat2,train_pred_lgbm1,train_pred_lgbm2,train_pred_xgb1
0,21.82535,22.50112,21.94135,22.80341,25.17577
1,32.08575,31.04042,33.88222,33.58557,28.23302
2,37.24590,38.44881,40.30427,36.28794,37.91516
3,45.30529,44.46108,43.66965,44.86389,44.94661
4,45.21940,44.77414,44.60095,41.66360,44.63388


In [52]:
preds_df.head()

,submission_cat1,submission_cat2,submission_lgbm1,submission_lgbm2,submission_xgb1
0,47.67475,44.18114,45.94662,45.91039,44.49270
1,17.88198,18.09641,18.78587,20.03686,18.57956
2,31.42835,32.43166,31.61498,32.80145,30.35761
3,46.61178,45.72016,45.40394,44.13508,45.07124
4,20.80786,20.47853,25.05543,27.44550,17.16118


In [53]:
type(preds_df)

pandas.core.frame.DataFrame

In [54]:
def run_lr(useful_features:List[str], TARGET:str, train_df:pd.DataFrame, test_df:pd.DataFrame) -> (List[float],List[float]):
    final_predictions = []
    scores = []

    kfold = model_selection.KFold(n_splits=Config.N_FOLDS, shuffle=True, random_state=Config.seed)

    for fold, (train_idx, valid_idx) in enumerate(kfold.split(train_df)):
        xtrain = train_df.iloc[train_idx].reset_index(drop=True)
        xvalid = train_df.iloc[valid_idx].reset_index(drop=True)

        xtest = test_df[useful_features].copy()

        ytrain = xtrain[TARGET]
        yvalid = xvalid[TARGET]

        xtrain = xtrain[useful_features]
        xvalid = xvalid[useful_features]

#         model = LogisticRegression()
        model = linear_model.LinearRegression()
        # Smaller C means more regularization; default=1.0
        # 2947.0517025518097
#         model = LogisticRegression(max_iter=500, C=2947.0517025518097, penalty='l2',solver='newton-cg')
#         model = LogisticRegression(C = 2947.0517025518097,
#                         max_iter = 500,
#                         penalty = 'l2',
#                         solver = 'liblinear')
        model.fit(xtrain, ytrain)

        preds_valid = model.predict_proba(xvalid)[:,-1]
        test_preds = model.predict_proba(xtest)[:,-1]

        final_predictions.append(test_preds)
#         score = metrics.roc_auc_score(yvalid, preds_valid)
        score = metrics.mean_squared_error(yvalid, preds_valid, squared=False)
        print(f"Fold={fold}, Score={score}")
        scores.append(score)
    return scores, final_predictions


In [55]:
# useful_features = ["pred_lda", "pred_gbc","pred_gbc2", "pred_cat_bp", "pred_cat1", "pred_lgbm1", "pred_lgbm2", "pred_lgbm_bp", "pred_xgb1", "pred_xgb_bp"]
useful_features = [ "train_pred_cat1", "train_pred_cat2", "train_pred_lgbm1", "train_pred_lgbm2", "train_pred_xgb1"]

In [56]:
oof_df[useful_features].head()

,train_pred_cat1,train_pred_cat2,train_pred_lgbm1,train_pred_lgbm2,train_pred_xgb1
0,21.82535,22.50112,21.94135,22.80341,25.17577
1,32.08575,31.04042,33.88222,33.58557,28.23302
2,37.24590,38.44881,40.30427,36.28794,37.91516
3,45.30529,44.46108,43.66965,44.86389,44.94661
4,45.21940,44.77414,44.60095,41.66360,44.63388


In [57]:
# preds_df[useful_features].head()

In [58]:
# fold_scores, final_predictions = run_lr(useful_features, TARGET, oof_df, preds_df)
# test_preds = np.mean(np.column_stack(final_predictions), axis=1)
# cv_score, std_dev = show_fold_scores(fold_scores)
# create_submission("level1_lr", TARGET, test_preds)

In [59]:
pd.options.display.max_colwidth = 100
pd.set_option("display.max_rows", 999)
pd.set_option("display.precision", 5)
pd.options.display.float_format = '{:.2f}'.format
pd.options.display.max_colwidth

100

In [60]:
all_cv_scores.sort_values(by=["Score"], ascending=False)

,Model,Score,StdDev,RunTime,n_estimators,n_folds,comments
12,lasso,11.33,0.30,0.96,500,10,
13,ridge,10.92,0.33,0.91,500,10,
14,ridge_50,10.92,0.33,0.91,500,10,
5,xgb2,9.07,0.33,83.73,500,10,
3,xgb3,9.06,0.35,95.93,500,10,
8,lgbm2,8.99,0.30,3.43,500,10,
4,xgb1,8.97,0.35,12.92,500,10,
6,lgbm0,8.94,0.30,8.55,500,10,
11,cat2,8.91,0.36,99.90,500,10,
7,lgbm1,8.90,0.33,8.79,500,10,


In [61]:
if Config.debug:
    print("*** Don't submit debug mode.  Only using partial training data ***")